In [1]:
import pandas as pd
import numpy as np

from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error

In [2]:
path = 'C:/Users/user/Desktop/PLoSONE/data/'

data = pd.read_csv(path + 'KU_dormitory_PLoS ONE.csv')
data.head()

,Year,Month,Day,Hour,Weekday,Hour_x,Hour_y,DOTW_x,DOTW_y,Holi,...,Humi,WS,THI,WCT,Cons_1,Holi_1,Cons_7,Holi_7,Cons_avg,Consumption
0,15,9,1,0,1,0.000000,1.000000,0.781831,0.62349,0,...,65.9,3.5,72.997827,26.544718,0.0,0,0.0,0,0.0,1198.80
1,15,9,1,1,1,0.258819,0.965926,0.781831,0.62349,0,...,65.6,3.9,72.529536,26.176736,0.0,0,0.0,0,0.0,1113.48
2,15,9,1,2,1,0.500000,0.866025,0.781831,0.62349,0,...,67.1,3.6,72.235089,25.871103,0.0,0,0.0,0,0.0,987.84
3,15,9,1,3,1,0.707107,0.707107,0.781831,0.62349,0,...,67.9,2.4,71.570456,25.472185,0.0,0,0.0,0,0.0,891.36
4,15,9,1,4,1,0.866025,0.500000,0.781831,0.62349,0,...,67.9,3.0,71.274014,25.169193,0.0,0,0.0,0,0.0,845.64


In [3]:
#data_drop = data.drop(columns = 'Date', axis = 1)
#data_drop.head()

In [4]:
x_train = data.iloc[:20472]
x_test = data.iloc[20472:]
y_train = data.iloc[:20472]

In [5]:
# target 정의 
y = y_train['Consumption']

In [6]:
# 데이터 셋 분리
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y, test_size=0.3, random_state=42)

In [7]:
# LightGBM
model = LGBMRegressor()

params = {'n_estimators': [1000, 1500],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [64],
    'subsample': [0.5],
    'colsample_bytree': [1.0],
    'boosting_type': ['gbdt', 'dart']
}

# GridSEarch
gr = GridSearchCV(model, param_grid = params, cv = 5)
gr.fit(X_train, Y_train)

print(gr.best_params_)

{'boosting_type': 'gbdt', 'colsample_bytree': 1.0, 'learning_rate': 0.01, 'n_estimators': 1000, 'num_leaves': 64, 'subsample': 0.5}


In [8]:
lgb = LGBMRegressor(random_state=42, boosting_type='gbdt', colsample_bytree=1.0,learning_rate=0.01, n_estimators=1000, 
                   num_leaves=64, subsample=0.5) # 다시 넣기
lgb.fit(X_train, Y_train)

y_pred = lgb.predict(X_valid)
y_pred

array([1151.9567167 , 1352.96197586, 1225.75908123, ..., 1397.88660588,
       1385.47270529, 1075.6573941 ])

In [9]:
# 실제값 
y_predict = lgb.predict(x_test)
y_predict.shape

(8760,)

In [10]:
y_predict

array([1064.08498182, 1080.71688722, 1097.00234195, ..., 1515.83766471,
       1402.72453423, 1327.89099982])

In [11]:
data

,Year,Month,Day,Hour,Weekday,Hour_x,Hour_y,DOTW_x,DOTW_y,Holi,...,Humi,WS,THI,WCT,Cons_1,Holi_1,Cons_7,Holi_7,Cons_avg,Consumption
0,15,9,1,0,1,0.000000,1.000000,0.781831,0.62349,0,...,65.9,3.5,72.997827,26.544718,0.00,0,0.00,0,0.000000,1198.80
1,15,9,1,1,1,0.258819,0.965926,0.781831,0.62349,0,...,65.6,3.9,72.529536,26.176736,0.00,0,0.00,0,0.000000,1113.48
2,15,9,1,2,1,0.500000,0.866025,0.781831,0.62349,0,...,67.1,3.6,72.235089,25.871103,0.00,0,0.00,0,0.000000,987.84
3,15,9,1,3,1,0.707107,0.707107,0.781831,0.62349,0,...,67.9,2.4,71.570456,25.472185,0.00,0,0.00,0,0.000000,891.36
4,15,9,1,4,1,0.866025,0.500000,0.781831,0.62349,0,...,67.9,3.0,71.274014,25.169193,0.00,0,0.00,0,0.000000,845.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29227,18,12,31,19,0,-0.965926,0.258819,0.000000,1.00000,0,...,42.8,1.6,36.355068,-2.389798,1442.16,1,1398.24,0,1590.994286,1558.80
29228,18,12,31,20,0,-0.866025,0.500000,0.000000,1.00000,0,...,38.6,0.6,36.369282,-1.009133,1405.08,1,1316.52,0,1530.565714,1538.28
29229,18,12,31,21,0,-0.707107,0.707107,0.000000,1.00000,0,...,44.2,1.0,34.240532,-2.932800,1464.84,1,1266.84,0,1480.371429,1513.44
29230,18,12,31,22,0,-0.500000,0.866025,0.000000,1.00000,0,...,45.6,1.3,32.967424,-4.326446,1443.60,1,1304.28,0,1463.400000,1400.76


In [12]:
df = pd.DataFrame({'Year' : data['Year'].iloc[20472:], 'Month' : data['Month'].iloc[20472:], 
              'Day' : data['Day'].iloc[20472:], 'Hour' : data['Hour'].iloc[20472:], 
              'Actual': data['Consumption'].iloc[20472:], 'Predict' : y_predict}).reset_index(drop=True)

df.to_csv('C:/Users/user/Desktop/PLoSONE/predict/KU_dormitory_PLoS_LightGBM.csv', header=True, index=False)